#### [Naver 리뷰 데이터셋 전처리] <hr>

-   train : https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
-   test : https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt


-   [1] 데이터 준비 <hr>


In [32]:
### => 데이터 다운로드
from urllib.request import urlretrieve

train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
filename = "../data/ratings_train.txt"

ret = urlretrieve(train_url, filename)


In [33]:
import pandas as pd

trainDF = pd.read_table(filename)


-   [2] 데이터 전처리 <hr>
    -   결측치, 중복 데이터
    -   클래스 균형 데이터 여부
    -   텍스트 데이터 전처리 => 정제(불용어, 노이즈 데이터 제거), 토큰화, 정제 ==> 단어사전
    -   텍스트 데이터 인코딩
    -   텍스트 데이터 패딩


-   [2-1] 기본 전처리 : 결측치, 중복데이터, 클래스 비율 체크, 피쳐와 타겟 분리


In [34]:
### ===> 불필요 컬럼 삭제
trainDF.drop(["id"], axis=1, inplace=True)

print(f"trainDF => {trainDF.shape}")


trainDF => (150000, 2)


In [35]:
### ===> 결측치 처리
trainDF.isnull().sum()
trainDF.dropna(inplace=True)

print(f"trainDF => {trainDF.shape}")


trainDF => (149995, 2)


In [36]:
### ===> 클래스 비율 체크
trainDF["label"].value_counts()


label
0    75170
1    74825
Name: count, dtype: int64

In [37]:
### ===> 피쳐와 레이블 분리
print(f"columns => {trainDF.columns}")

documentDF = trainDF[["document"]]
labelSR = trainDF["label"]

print(f"documentDF => {documentDF.shape}, labelSR => {labelSR.shape}")


columns => Index(['document', 'label'], dtype='object')
documentDF => (149995, 1), labelSR => (149995,)


-   [2-2] 텍스트 데이터 전처리 : 정제(노이즈 데이터, 불용어 제거), 토큰화, 정수화


In [38]:
### ===> 노이즈 데이터 기준 : 한글만, 한글+영어, 한글+영어+숫자


In [39]:
import string

stop_word = string.punctuation
hangule_patten = "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"


In [40]:
# # 한글과 공백을 제외하고 모두 제거
# documentDF['document'] = documentDF['document'].str.replace(hangule_patten, '')
# documentDF.head()


In [41]:
hangule_stopword = "https://gist.githubusercontent.com/chulgil/d10b18575a73778da4bc83853385465c/raw/a1a451421097fa9a93179cb1f1f0dc392f1f9da9/stopwords.txt"

urlretrieve(hangule_stopword, "../data/stopwords.txt")


('../data/stopwords.txt', <http.client.HTTPMessage at 0x208f18a24f0>)

In [42]:
import string, re
import numpy as np


def perprocessing(text: str, replace=np.nan):
    stop_words = "ㅃㅉㄸㄲㅆㅂㅈㄷㄱㅅㅛㅕㅑㅐㅔㅁㄴㅇㄹㅎㅗㅓㅏㅣㅋㅌㅊㅍㅠㅜㅡ"
    text = re.sub(hangule_patten, "", text)
    removed = re.sub(f"[{string.punctuation}a-zA-Z\\d{stop_words}]", "", text).strip()
    replaced = replace if removed == "" else removed
    return replaced


In [43]:
trainDF["document"] = trainDF["document"].apply(perprocessing)


In [44]:
trainDF.dropna(inplace=True)
trainDF.reset_index(drop=True, inplace=True)


In [45]:
documentDF = trainDF[["document"]]
labelSR = trainDF["label"]

print(f"documentDF => {documentDF.shape}, labelSR => {labelSR.shape}")


documentDF => (148389, 1), labelSR => (148389,)


In [46]:
documentDF


,document
0,아 더빙 진짜 짜증나네요 목소리
1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 솔직히 재미는 없다평점 조정
4,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
...,...
148384,인간이 문제지 소는 뭔죄인가
148385,평점이 너무 낮아서
148386,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
148387,청춘 영화의 최고봉방황과 우울했던 날들의 자화상


-   [2-3] 토큰화 <hr>


In [47]:
from konlpy.tag import Okt


In [48]:
# 형태소 분석기 인스턴스 생성
okt = Okt()


In [49]:
vocab = {}
for idx in range(documentDF.shape[0]):
    result = okt.morphs(documentDF.iloc[idx][0])
    for word in result:
        if len(word) > 1:
            if word in vocab.keys():
                vocab[word] += 1
            else:
                vocab[word] = 1


In [50]:
with open("../data/hangul_stopword.txt", "r", encoding="utf-8") as f:
    hangule_stopwords = f.read().replace("\n", ",").split(",")


In [51]:
hangule_stopwords += ["보고", "하는", "하고", "그냥", "보다", "없는", "봤는데"]


In [52]:
filtered_vocab = {
    word: count for word, count in vocab.items() if word not in hangule_stopwords
}


In [53]:
len(filtered_vocab)


97373

In [54]:
sorted(filtered_vocab.items(), key=lambda x: x[1], reverse=True)


[('영화', 50647),
 ('너무', 11150),
 ('정말', 9752),
 ('진짜', 8336),
 ('연기', 6328),
 ('평점', 6315),
 ('최고', 6041),
 ('스토리', 5334),
 ('생각', 5313),
 ('드라마', 5062),
 ('감동', 4882),
 ('사람', 4837),
 ('배우', 4291),
 ('감독', 4118),
 ('재미', 3919),
 ('내용', 3809),
 ('쓰레기', 3564),
 ('작품', 2980),
 ('사랑', 2943),
 ('없다', 2831),
 ('다시', 2813),
 ('마지막', 2757),
 ('좋은', 2663),
 ('이건', 2604),
 ('정도', 2586),
 ('같은', 2554),
 ('완전', 2540),
 ('입니다', 2531),
 ('있는', 2509),
 ('처음', 2453),
 ('장면', 2425),
 ('액션', 2398),
 ('주인공', 2375),
 ('이렇게', 2329),
 ('보는', 2294),
 ('최악', 2275),
 ('이야기', 2174),
 ('지금', 2170),
 ('별로', 2132),
 ('봐도', 2123),
 ('느낌', 2102),
 ('연출', 2076),
 ('인데', 2062),
 ('없고', 2059),
 ('명작', 2040),
 ('역시', 1995),
 ('많이', 1933),
 ('재밌게', 1913),
 ('이해', 1898),
 ('이영화', 1817),
 ('라고', 1792),
 ('보면', 1791),
 ('이나', 1765),
 ('때문', 1738),
 ('여자', 1730),
 ('해서', 1655),
 ('아깝다', 1634),
 ('보기', 1613),
 ('이고', 1605),
 ('기억', 1589),
 ('결말', 1580),
 ('마음', 1552),
 ('인생', 1542),
 ('소재', 1508),
 ('없이', 1456),
 ('수준', 1450

In [76]:
vocabDF = pd.Series(filtered_vocab).to_frame()
vocabDF.sort_values(by=0, ascending=False, inplace=True)
vocabDF


,0
영화,50647
너무,11150
정말,9752
진짜,8336
연기,6328
...,...
화한다는,1
데빈,1
익부,1
가야지다,1


In [79]:
VOCAB_DICT = {0: "<UNK>", 1: "<PAD>"}

for idx in range(1000):
    VOCAB_DICT[idx + 2] = sorted(
        filtered_vocab.items(), key=lambda x: x[1], reverse=True
    )[idx][0]


In [80]:
VOCAB_DICT


{0: '<UNK>',
 1: '<PAD>',
 2: '영화',
 3: '너무',
 4: '정말',
 5: '진짜',
 6: '연기',
 7: '평점',
 8: '최고',
 9: '스토리',
 10: '생각',
 11: '드라마',
 12: '감동',
 13: '사람',
 14: '배우',
 15: '감독',
 16: '재미',
 17: '내용',
 18: '쓰레기',
 19: '작품',
 20: '사랑',
 21: '없다',
 22: '다시',
 23: '마지막',
 24: '좋은',
 25: '이건',
 26: '정도',
 27: '같은',
 28: '완전',
 29: '입니다',
 30: '있는',
 31: '처음',
 32: '장면',
 33: '액션',
 34: '주인공',
 35: '이렇게',
 36: '보는',
 37: '최악',
 38: '이야기',
 39: '지금',
 40: '별로',
 41: '봐도',
 42: '느낌',
 43: '연출',
 44: '인데',
 45: '없고',
 46: '명작',
 47: '역시',
 48: '많이',
 49: '재밌게',
 50: '이해',
 51: '이영화',
 52: '라고',
 53: '보면',
 54: '이나',
 55: '때문',
 56: '여자',
 57: '해서',
 58: '아깝다',
 59: '보기',
 60: '이고',
 61: '기억',
 62: '결말',
 63: '마음',
 64: '인생',
 65: '소재',
 66: '없이',
 67: '수준',
 68: '현실',
 69: '한번',
 70: '가장',
 71: '매력',
 72: '한다',
 73: '반전',
 74: '전개',
 75: '라는',
 76: '한국',
 77: '남자',
 78: '가슴',
 79: '되는',
 80: '음악',
 81: '하게',
 82: '원작',
 83: '만든',
 84: '인간',
 85: '추천',
 86: '눈물',
 87: '않고',
 88: '인지',
 89: '모든',
 90

In [81]:
test = ["영화", "최고", "&", "생각"]
sentence = []
for tt in test:
    result = next((k for k, v in VOCAB_DICT.items() if v == tt), 0)
    sentence.append(result)
sentence


[2, 8, 0, 10]

In [82]:
decoding = []
words = []

for tt in sentence:
    words.append(VOCAB_DICT.get(tt))

words


['영화', '최고', '<UNK>', '생각']